<a href="https://colab.research.google.com/github/Guhenriquetorres/Analise_Dados_Enem_2021/blob/main/Analise_Meteorologia_Brasilia_MapReduce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***
## **<center>Análise do histórico de dados meteorológicos de Brasília utilizando MapReduce</center>**


**<center>Processamento de Dados Massivos</center>**
***

<br>

Como forma de cristalizar o aprendizado sobre o modelo de programação MapReduce, será explorado um tipo de dado que fatalmente poderia ser classificado por massivo. Trata-se dos registros de dados meteorológicos captados por sensores que são utilizados para a obtenção das informações de previsão do tempo.

Os dados foram obtidos em https://portal.inmet.gov.br/dadoshistoricos e consolidados antes de serem disponibilizados para o exercício.

Lista de ações e perguntas a ser consideradas para avaliação:

* Análise exploratória mínima sobre os dados;
* Qual foi a maior temperatura já registrada em Brasília entre os dados disponíveis?
* Qual foi o dia em que essa temperatura foi alcançada?
* Qual seria temperatura média anual?
* Qual o dia em que foi registrada a maior quantidade de chuva, segundo os dados?
* Qual o mês com maior média em que foi registrada a maior quantidade de chuva, segundo os dados?

**DETALHE IMPORTANTE**:

DEVE SER UTILIZADO O MODELO DE PROGRAMAÇÂO **MapReduce**, CONFORME TRABALHADO NAS AULAS, OU SEJA, DEVEM SER EMPREGADAS AS FUNÇÕES MAP, SHUFFLE E REDUCE.

### Análise exploratória

In [72]:
import pandas as pd

df = pd.read_csv('https://github.com/alexvaroz/data_science_alem_do_basico/raw/master/historico_meteorologico_brasilia.csv')

In [73]:
df.head()

,DATA,HORA,PRECIPITACAO_HORARIO_mm,TEMPERATURA,UMIDADE_RELATIVA_AR,ANO,MES,DIA,DIA_MES
0,2015-10-16,2,0.0,22.6,40.0,2015,10,16,16_10
1,2016-06-24,9,0.0,13.0,77.0,2016,6,24,24_6
2,2001-01-31,6,0.0,20.1,82.0,2001,1,31,31_1
3,2015-09-20,13,0.0,28.3,14.0,2015,9,20,20_9
4,2004-08-10,21,0.0,21.8,41.0,2004,8,10,10_8


In [74]:
print(df['TEMPERATURA'].max())
print(df['TEMPERATURA'].mean())

35.5
21.363354837411055


### Questões a serem respondidas

In [75]:
import requests

request_content = requests.get('https://github.com/alexvaroz/data_science_alem_do_basico/raw/master/historico_meteorologico_brasilia.csv')
arquivo_base = open("dados_meteorologicos.csv", 'w')
arquivo_base.write(str(request_content.text))
arquivo_base.close()

In [81]:
from functools import reduce

def map_meteorologicos_bsb(nome_arquivo_entrada):
    arquivo_entrada = open(nome_arquivo_entrada, 'r')
    primeira_linha = arquivo_entrada.readline()  # para eliminar a primeira linha

    # Inicializa um dicionário para rastrear a soma de chuva por mês
    mes_soma_chuva = {}

    for linha in arquivo_entrada:
        partes = linha.split(',')
        if len(partes) >= 4:
            data = partes[0].strip()
            mes = data.split('-')[1]  # Extrai o mês da data
            valor = partes[2].strip()  # Valor da chuva
            if valor:  # Verifique se a string não está vazia
                try:
                    valor = float(valor)  # Tenta converter para float
                    # Atualiza o dicionário com a soma de chuva por mês
                    if mes in mes_soma_chuva:
                        mes_soma_chuva[mes].append(valor)
                    else:
                        mes_soma_chuva[mes] = [valor]
                except ValueError:
                    pass  # Ignora valores não numéricos
    arquivo_entrada.close()

    # Calcula a média de chuva para cada mês e rastreia o mês com a maior média
    mes_maior_media = None
    maior_media = 0

    for mes, chuvas in mes_soma_chuva.items():
        media = reduce(lambda x, y: x + y, chuvas) / len(chuvas)
        if media > maior_media:
            maior_media = media
            mes_maior_media = mes

    if mes_maior_media is None:
        mes_maior_media = "N/A"  # Define como "N/A" se não houver valores válidos

    return mes_maior_media

nome_arquivo_entrada = 'dados_meteorologicos.csv'
mes_com_maior_media_chuva = map_meteorologicos_bsb(nome_arquivo_entrada)
print(f"Mês com a maior média de chuva: {mes_com_maior_media_chuva}")


Mês com a maior média de chuva: 11


In [91]:
def map_meteorologicos_bsb(nome_arquivo_entrada, nome_arquivo_saida):
    arquivo_entrada = open(nome_arquivo_entrada, 'r')
    primeira_linha = arquivo_entrada.readline()  # para eliminar a primeira linha
    max_temperatura = float('-inf')  # Inicializa o máximo com um valor negativo infinito

    for linha in arquivo_entrada:
        partes = linha.split(',')
        if len(partes) >= 4:
            valor = partes[3].strip()
            if valor:  # Verifique se a string não está vazia
                try:
                    valor = float(valor)  # Tenta converter para float
                    if valor > max_temperatura:
                        max_temperatura = valor  # Atualiza o máximo se encontrar um valor maior
                except ValueError:
                    pass  # Ignora valores não numéricos
    arquivo_entrada.close()

    if max_temperatura == float('-inf'):
        max_temperatura = 0  # Define o máximo como 0 se não houver valores válidos na coluna

    # Escreve o maior valor de temperatura no arquivo de saída
    with open(nome_arquivo_saida, 'w') as arquivo_saida:
        arquivo_saida.write(f"Maior temperatura: {max_temperatura}")

nome_arquivo_entrada = 'dados_meteorologicos.csv'
nome_arquivo_saida = 'saida_maior_temperatura.txt'
map_meteorologicos_bsb(nome_arquivo_entrada, nome_arquivo_saida)




In [93]:
def map_meteorologicos_bsb(nome_arquivo_entrada, nome_arquivo_saida):
    arquivo_entrada = open(nome_arquivo_entrada, 'r')
    primeira_linha = arquivo_entrada.readline()  # para eliminar a primeira linha
    max_temperatura = float('-inf')  # Inicializa a temperatura máxima com um valor negativo infinito
    data_max_temperatura = None  # Inicializa a data correspondente como None

    for linha in arquivo_entrada:
        partes = linha.split(',')
        if len(partes) >= 4:
            data = partes[0].strip()
            valor = partes[3].strip()
            if valor:  # Verifique se a string não está vazia
                try:
                    valor = float(valor)  # Tenta converter para float
                    if valor > max_temperatura:
                        max_temperatura = valor  # Atualiza a temperatura máxima
                        data_max_temperatura = data  # Atualiza a data correspondente
                except ValueError:
                    pass  # Ignora valores não numéricos
    arquivo_entrada.close()

    if max_temperatura == float('-inf'):
        max_temperatura = 0

    if data_max_temperatura is None:
        data_max_temperatura = "N/A"

    with open(nome_arquivo_saida, 'w') as arquivo_saida:
        arquivo_saida.write(f"Maior temperatura: {max_temperatura}\n")
        arquivo_saida.write(f"Data correspondente: {data_max_temperatura}")

nome_arquivo_entrada = 'dados_meteorologicos.csv'
nome_arquivo_saida = 'saida_maior_temperatura_data.txt'
map_meteorologicos_bsb(nome_arquivo_entrada, nome_arquivo_saida)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [79]:
def map_meteorologicos_bsb(nome_arquivo_entrada, nome_arquivo_saida):
    arquivo_entrada = open(nome_arquivo_entrada, 'r')
    primeira_linha = arquivo_entrada.readline()
    max_chuva = float('-inf')
    data_max_chuva = None

    for linha in arquivo_entrada:
        partes = linha.split(',')
        if len(partes) >= 4:
            data = partes[0].strip()
            valor = partes[2].strip()
            if valor:
                try:
                    valor = float(valor)
                    if valor > max_chuva:
                        max_chuva = valor
                        data_max_chuva = data
                except ValueError:
                    pass
    arquivo_entrada.close()

    if max_chuva == float('-inf'):
        max_chuva = 0
    if data_max_chuva is None:
        data_max_chuva = "N/A"

    with open(nome_arquivo_saida, 'w') as arquivo_saida:
        arquivo_saida.write(f"Maior chuva: {max_chuva}\n")
        arquivo_saida.write(f"Data correspondente: {data_max_chuva}")

nome_arquivo_entrada = 'dados_meteorologicos.csv'
nome_arquivo_saida = 'saida_maior_chuva_data.txt'
map_meteorologicos_bsb(nome_arquivo_entrada, nome_arquivo_saida)


In [80]:
def map_meteorologicos_bsb(nome_arquivo_entrada, nome_arquivo_saida):
    arquivo_entrada = open(nome_arquivo_entrada, 'r')
    primeira_linha = arquivo_entrada.readline()  # para eliminar a primeira linha
    mes_media_chuva = {}  # Dicionário para rastrear a média de chuva por mês

    for linha in arquivo_entrada:
        partes = linha.split(',')
        if len(partes) >= 4:
            data = partes[0].strip()
            mes = data.split('-')[1]  # Extrai o mês da data
            valor = partes[2].strip()  # Valor da chuva
            if valor:  # Verifique se a string não está vazia
                try:
                    valor = float(valor)
                    if mes in mes_media_chuva:
                        mes_media_chuva[mes][0] += valor
                        mes_media_chuva[mes][1] += 1
                    else:
                        mes_media_chuva[mes] = [valor, 1]
                except ValueError:
                    pass  # Ignora valores não numéricos
    arquivo_entrada.close()

    mes_maior_media = None
    maior_media = 0
    for mes, (soma_chuva, contador) in mes_media_chuva.items():
        media = soma_chuva / contador
        if media > maior_media:
            maior_media = media
            mes_maior_media = mes

    if mes_maior_media is None:
        mes_maior_media = "N/A"

    with open(nome_arquivo_saida, 'w') as arquivo_saida:
        arquivo_saida.write(f"Mês com a maior média de chuva: {mes_maior_media}")

nome_arquivo_entrada = 'dados_meteorologicos.csv'
nome_arquivo_saida = 'saida_maior_media_chuva.txt'
map_meteorologicos_bsb(nome_arquivo_entrada, nome_arquivo_saida)
